In [1]:
import numpy as np
from tabulate import tabulate

In [2]:
def input_matrix(n):
    a_matrix=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            a_matrix[i,j]=input(f"Coeff for {i},{j}: ")
    b_matrix=np.zeros((n,1))
    for i in range(n):
        b_matrix[i,0]=input(f"RHS Value for {i}: ")
    return a_matrix,b_matrix

In [3]:
def gauss_jacobi(n, a_matrix, b_matrix,x_i, iterations, tolerance, mode):
    table_data = []  
    prev_x = np.copy(x_i)
    if mode==1:
        for z in range(iterations):
            all_coeff = -np.matmul(a_matrix, x_i)
            for i in range(n):
                all_coeff[i] += a_matrix[i, i] * x_i[i]
            new_x = np.add(b_matrix, all_coeff)
            for i in range(n):
                new_x[i] /= a_matrix[i, i]
            table_data.append([z + 1] + list(new_x))

            prev_x = np.copy(new_x)
            x_i = new_x

    else:
        z=0
        while True:
            all_coeff = -np.matmul(a_matrix, x_i)
            for i in range(n):
                all_coeff[i] += a_matrix[i, i] * x_i[i]
            new_x = np.add(b_matrix, all_coeff)
            for i in range(n):
                new_x[i] /= a_matrix[i, i]
            table_data.append([z + 1] + list(new_x))
            if np.allclose(new_x, prev_x, atol=tolerance):
                break
            z+=1    
            prev_x = np.copy(new_x)
            x_i = new_x

    headers = ["Iteration"] + [f"X{i+1}" for i in range(n)]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

In [14]:
def gauss_jacobi_matrix(n, A,b,x_i,iterations, tol, mode):
    table_data=[]
    Diag=np.diag(A)
    D=np.diagflat(Diag)
    D_inv=np.linalg.inv(D)
    R=A-D
    T=-np.matmul(D_inv,R)
    C=np.matmul(D_inv,b)
    x=x_i
    if mode==1:
        for i in range(iterations):
            x_old = x
            x = np.matmul(T,x) + C
            x_new = x
            table_data.append([i + 1] + list(x_new))
    else: 
        i = 0
        while True:
            x_old = x.copy() 
            x = np.matmul(T, x) + C 
            x_new = x 
            table_data.append([i + 1] + list(x_new)) 
            
            if np.linalg.norm(x_new - x_old) <= tol:
                break
            i += 1
            
    headers = ["Iteration"] + [f"X{i+1}" for i in range(n)]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))       

In [5]:
def pre_iter(n):
    a_matrix,b_matrix=input_matrix(n)
    x_i=np.zeros((n,1))
    for i in range(n):
        x_i[i,0]=input(f"Initial Guess for x{i+1}")
    iterations=int(input("No. of iter:{if tol mode enter 0} "))
    tol=int(input("No. of decimal place accuracy:{if iter mode enter 0} "))
    tol=10**(-tol)
    return x_i, a_matrix, b_matrix, iterations, tol

In [6]:
x_i, a_matrix, b_matrix, iterations, tol= pre_iter(3)

Coeff for 0,0:  12
Coeff for 0,1:  3
Coeff for 0,2:  -5
Coeff for 1,0:  1
Coeff for 1,1:  5
Coeff for 1,2:  3
Coeff for 2,0:  3
Coeff for 2,1:  7
Coeff for 2,2:  13
RHS Value for 0:  1
RHS Value for 1:  28
RHS Value for 2:  76
Initial Guess for x1 1
Initial Guess for x2 0
Initial Guess for x3 1
No. of iter:{if tol mode enter 0}  6
No. of decimal place accuracy:{if iter mode enter 0}  3


In [40]:
#mode 1=iterations, mode 2=tolerance
mode=1
gauss_jacobi(3, a_matrix, b_matrix,x_i, iterations, tol, mode)

+-------------+----------+---------+---------+
|   Iteration |       X1 |      X2 |      X3 |
+=============+==========+=========+=========+
|           1 | 0.5      | 4.8     | 5.61538 |
+-------------+----------+---------+---------+
|           2 | 1.22308  | 2.13077 | 3.14615 |
+-------------+----------+---------+---------+
|           3 | 0.861538 | 3.46769 | 4.41657 |
+-------------+----------+---------+---------+
|           4 | 1.05665  | 2.77775 | 3.78012 |
+-------------+----------+---------+---------+
|           5 | 0.963945 | 3.1206  | 4.1066  |
+-------------+----------+---------+---------+
|           6 | 1.01427  | 2.94325 | 3.94338 |
+-------------+----------+---------+---------+


In [41]:
gauss_jacobi_matrix(3, a_matrix, b_matrix,x_i, iterations, tol, mode)

+-------------+----------+---------+---------+
|   Iteration |       X1 |      X2 |      X3 |
+=============+==========+=========+=========+
|           1 | 0.5      | 4.8     | 5.61538 |
+-------------+----------+---------+---------+
|           2 | 1.22308  | 2.13077 | 3.14615 |
+-------------+----------+---------+---------+
|           3 | 0.861538 | 3.46769 | 4.41657 |
+-------------+----------+---------+---------+
|           4 | 1.05665  | 2.77775 | 3.78012 |
+-------------+----------+---------+---------+
|           5 | 0.963945 | 3.1206  | 4.1066  |
+-------------+----------+---------+---------+
|           6 | 1.01427  | 2.94325 | 3.94338 |
+-------------+----------+---------+---------+


In [34]:
def gauss_siedel(n, A, b,x_i, iterations, tolerance):
    table_data=[]
    L = np.tril(A)
    U = np.triu(A,1)
    L_inv = np.linalg.inv(L)
    T = -np.matmul(L_inv,U)
    C = np.matmul(L_inv,b)
    x = x_i
    
    for i in range(iterations):
        x_old = x
        x = np.matmul(T,x) + C
        x_new = x
        table_data.append([i + 1] + list(x_new))
        if np.allclose(x_new, x_old, atol=tolerance):
            break
    headers = ["Iteration"] + [f"X{i+1}" for i in range(n)]
    print(tabulate(table_data, headers=headers, tablefmt="grid")) 
    return x

In [39]:
m=gauss_siedel(3,a_matrix,b_matrix,x_i,6,1/np.inf)

+-------------+----------+---------+---------+
|   Iteration |       X1 |      X2 |      X3 |
+=============+==========+=========+=========+
|           1 | 0.5      | 4.9     | 3.09231 |
+-------------+----------+---------+---------+
|           2 | 0.146795 | 3.71526 | 3.81176 |
+-------------+----------+---------+---------+
|           3 | 0.742751 | 3.1644  | 3.97084 |
+-------------+----------+---------+---------+
|           4 | 0.946753 | 3.02814 | 3.99713 |
+-------------+----------+---------+---------+
|           5 | 0.99177  | 3.00337 | 4.00009 |
+-------------+----------+---------+---------+
|           6 | 0.999195 | 3.00011 | 4.00013 |
+-------------+----------+---------+---------+
